In [1]:
import numpy as np
import pandas as pd
import os
import pandas as pd
import sys

# Config

In [2]:
HYPER_OPT = True 
HYPER_OPT_TIME = 3600*11
USE_ORIGINAL_DATA = True 
SUBMIT_TO_KAGGLE = False

In [3]:
# Define the base directory (where the notebook is running)
base_dir = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Define the data directory
data_dir = os.path.join(base_dir, "data")

model_dir = os.path.join(base_dir, "models")

sys.path.append(base_dir)

# Data Extraction

In [4]:

# # Construct file paths
# train_file = os.path.join(data_dir, "train.csv")
# test_file = os.path.join(data_dir, "test.csv")
# original_file = os.path.join(data_dir, "Insurance Premium Prediction Dataset.csv")

# # Load the datasets
# train_df = pd.read_csv(train_file, index_col="id")
# test_df = pd.read_csv(test_file, index_col="id")
# original_df = pd.read_csv(original_file)

In [5]:
from axyom_utilities.data_extraction import extract_data

X_train, y_train, X_test, X_orig, y_orig = extract_data(data_dir, USE_ORIGINAL_DATA)

# Data Cleaning

In [6]:
from axyom_utilities.preprocessing import preprocess 

#original_df = original_df.dropna(subset=["Premium Amount"])

# train_df = preprocess(train_df)
# test_df = preprocess(test_df)
# original_df = preprocess(original_df)

X_train = preprocess(X_train)
X_test = preprocess(X_test)
if USE_ORIGINAL_DATA:
    X_orig = preprocess(X_orig)

# HyperOpt

In [7]:
import json
from axyom_utilities.wrappers import XGBRegressorWrapper
from axyom_utilities.training import train_model_cv
import optuna
import torch
from optuna.samplers import TPESampler
from optuna.visualization.matplotlib import (
    plot_optimization_history, 
    plot_param_importances, 
    plot_parallel_coordinate,
    plot_slice,
    plot_contour
)
import matplotlib.pyplot as plt

if HYPER_OPT:
    fixed_params = {
        "n_estimators": 10000,
        "objective": "reg:squarederror",  # XGBoost regression objective
        "tree_method": "gpu_hist" if torch.cuda.is_available() else "auto",
        "verbosity": 0,
        "enable_categorical": True
    }
    
    # Define the Optuna objective function
    def objective(trial):
        # Define hyperparameter space
        varying_params = {
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 8, 15),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 50, log=True),
            "subsample": trial.suggest_float("subsample", 0.4, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "lambda": trial.suggest_float("lambda", 1e-3, 10, log=True),
            "alpha": trial.suggest_float("alpha", 1e-3, 10, log=True)
        }

        xgb_params = {**fixed_params, **varying_params}
        
        # Initialize XGBRegressor with trial parameters
        model = XGBRegressorWrapper(**xgb_params)
        
        # Evaluate using K-Fold CV with early stopping
        results = train_model_cv(\
            model, 
            X_train, 
            y_train, 
            X_test, 
            X_orig,
            y_orig,
            cv_splits=5, 
            early_stopping_rounds=50
        )
        score = results['cv_scores'].mean()

        trial.set_user_attr("best_iteration", results['best_iteration'])
        
        return score
    
    # Prepare data
    # Replace X_train, y_train, and X_test with your data
    # Example:
    # X_train, X_test, y_train = ...
    
    # Run Optuna optimization
    study = optuna.create_study( \
        direction="minimize", 
        study_name="XGB_v1", 
        storage="sqlite:///xgb_study_v1.db", 
        load_if_exists=True,
        sampler=TPESampler(seed=666)
    )
    study.optimize(objective, n_trials=100, timeout=HYPER_OPT_TIME)
    
    # Best parameters and result
    print("Best Trial: ", study.best_trial.params)
    print("Best RMSE: ", study.best_value)

    xgb_best_params = {**fixed_params, **study.best_trial.params}

    xgb_best_params["n_estimators"] = study.best_trial.user_attrs.get("best_iteration", None)

    with open("xgb_best_params.json", "w") as f:
        json.dump(xgb_best_params, f, indent=4)
   
    plot_optimization_history(study)
    plt.show()
    
    plot_param_importances(study)
    plt.show()
    
    plot_slice(study)
    plt.show()

else:
    xgb_best_params = {
        'n_estimators': 2225,
        'objective': 'reg:squarederror',
        'tree_method': 'gpu_hist',
        'verbosity': 0,
        'enable_categorical': True,
        'learning_rate': 0.003059929305190928,
        'max_depth': 8,
        'min_child_weight': 12.496270561250991,
        'subsample': 0.8428246186530037,
        'colsample_bytree': 0.9999895920675128,
        'gamma': 2.937438656382514,
        'lambda': 1.5752155403171972,
        'alpha': 0.4038060866963702
    }


[I 2024-12-16 21:51:16,367] Using an existing study with name 'XGB_v1' instead of creating a new one.


Training fold 1...
Fold 1 RMSE: 1.0499
Training fold 2...
Fold 2 RMSE: 1.0488
Training fold 3...
Fold 3 RMSE: 1.0473
Training fold 4...
Fold 4 RMSE: 1.0503
Training fold 5...


[I 2024-12-16 21:53:29,427] Trial 2 finished with value: 1.048994163843117 and parameters: {'learning_rate': 0.05433350003437882, 'max_depth': 14, 'min_child_weight': 1.509804001525835, 'subsample': 0.8367148343488449, 'colsample_bytree': 0.9708747744678036, 'gamma': 0.06351598517383816, 'lambda': 0.04511809923805299, 'alpha': 0.0015676574662308934}. Best is trial 2 with value: 1.048994163843117.


Fold 5 RMSE: 1.0487
Mean CV RMSE: 1.0490 ± 0.0010
Training fold 1...
Fold 1 RMSE: 1.0488
Training fold 2...
Fold 2 RMSE: 1.0477
Training fold 3...
Fold 3 RMSE: 1.0463
Training fold 4...
Fold 4 RMSE: 1.0488
Training fold 5...
Fold 5 RMSE: 1.0475
Mean CV RMSE: 1.0478 ± 0.0009


[I 2024-12-16 22:26:00,832] Trial 3 finished with value: 1.0478290056562236 and parameters: {'learning_rate': 0.001768215377734423, 'max_depth': 12, 'min_child_weight': 0.008728853020978604, 'subsample': 0.8464925012884582, 'colsample_bytree': 0.5157352018243503, 'gamma': 3.5042237608299063, 'lambda': 0.014890607747597583, 'alpha': 1.2529040614881077}. Best is trial 3 with value: 1.0478290056562236.


...

Fold 5 RMSE: 1.0452
Mean CV RMSE: 1.0454 ± 0.0009
Training fold 1...
Fold 1 RMSE: 1.0463
Training fold 2...
Fold 2 RMSE: 1.0452
Training fold 3...
Fold 3 RMSE: 1.0438
Training fold 4...
Fold 4 RMSE: 1.0464
Training fold 5...


[I 2024-12-17 01:55:27,779] Trial 34 finished with value: 1.0453818789183145 and parameters: {'learning_rate': 0.0021978412286356554, 'max_depth': 9, 'min_child_weight': 13.769796614512282, 'subsample': 0.8201686655552866, 'colsample_bytree': 0.9517316594253322, 'gamma': 4.196887300383792, 'lambda': 0.1770413282973609, 'alpha': 0.11609821800436655}. Best is trial 25 with value: 1.0453613381313727.


Fold 5 RMSE: 1.0452
Mean CV RMSE: 1.0454 ± 0.0009
Training fold 1...
Fold 1 RMSE: 1.0465
Training fold 2...
Fold 2 RMSE: 1.0453
Training fold 3...
Fold 3 RMSE: 1.0440
Training fold 4...
Fold 4 RMSE: 1.0466
Training fold 5...


[W 2024-12-17 02:03:58,323] Trial 35 failed with parameters: {'learning_rate': 0.0014055817540153878, 'max_depth': 8, 'min_child_weight': 9.001701045150105, 'subsample': 0.8123380792736161, 'colsample_bytree': 0.8764072067850127, 'gamma': 3.55474318759224, 'lambda': 0.21305813081580105, 'alpha': 0.6444099583891106} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Axyom\anaconda3\envs\kaggle_env\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Axyom\AppData\Local\Temp\ipykernel_10324\762608703.py", line 45, in objective
    results = train_model_cv(\
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insurance\axyom_utilities\training.py", line 32, in train_model_cv
    model.fit(
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insurance\axyom_utilities\wrappers.py", line 41, in fit
    self.xgb_model_.fit(
  File "c:\Users\Axyom\anaconda3\envs\kaggle_env\

KeyboardInterrupt: 

In [ ]:
xgb_best_params

In [ ]:
xgb_best_params["n_estimators"] = 10000

# Model Training

In [ ]:
# Initialize a CatBoost Regressor
model = XGBRegressorWrapper(**xgb_best_params)

# Use the train_model function to train and evaluate the model
results = train_model_cv(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    X_orig=X_orig,
    y_orig=y_orig,
    early_stopping_rounds=100,
    cv_splits=7
)

In [ ]:
mean_score = np.mean(results["cv_scores"])
std_score = np.std(results["cv_scores"])

# Prepare the data
data = {
    "mean_score": mean_score,
    "std_score": std_score
}

# Save to a JSON file
with open("score.json", "w") as json_file:
    json.dump(data, json_file, indent=4)  # Use `indent` for readability

In [ ]:
import joblib

# Save OOF predictions as a CSV file
oof_preds_df = pd.DataFrame({"oof_preds": results["oof_preds"]})
oof_preds_df.to_csv("oof_preds", index=False)
print("OOF predictions saved to oof_preds.csv.")

test_preds_df = pd.DataFrame({"test_preds": results["test_preds"]})
test_preds_df.to_csv("test_preds", index=False)
print("Test predictions saved to test_preds.csv.")

joblib.dump(results["models"], "xgboost_models.pkl")

# Submission

In [ ]:
y_pred = np.expm1(results['test_preds'])

submission = pd.DataFrame({
    'id': X_test.index,  
    'Premium Amount': y_pred
})

submission.to_csv('submission.csv', index=False)

In [ ]:
# import os
# from IPython.display import display, Javascript

# # Define your message and file paths
# COMP_NAME = "playground-series-s4e12"
# FILE_PATH = "submission.csv"

# SUBMIT_MESSAGE = f"Clean XGB: Mean score: {mean_score:.4f} +/- {std_score:.4f}"

# # Submit to Kaggle
# if SUBMIT_TO_KAGGLE: 
#     os.system(f'kaggle competitions submit -c {COMP_NAME} -f {FILE_PATH} -m "{SUBMIT_MESSAGE}"')

# # Git commit and push
# GIT_COMMIT_MESSAGE = f"Submission: {SUBMIT_MESSAGE}"

# # save notebook
# # display(Javascript('IPython.notebook.save_checkpoint()'))

# # Commands for Git
# os.system("git add .")  # Stage all changes (adjust if you only want specific files)
# os.system(f'git commit -m "{GIT_COMMIT_MESSAGE}"')  # Commit changes with a message
# os.system("git push origin main")  # Push to the main branch (change branch if needed)